In [4]:
import pandas as pd
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma


CHROMA_PATH = "chroma"
DATA_PATH = "data"

In [2]:
data = pd.read_csv('../ai_medreview/data/data.csv')
data.head()

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id
0,2023-07-13 15:28:00,Extremely likely,Very friendly caring staff treated me with res...,NaN,Earls-Court-Surgery,Brompton-Health-PCN,positive,0.892932,neutral,0.000000,13.0,3,5.0,Staff Empathy and Compassion,NaN,NaN,NaN
1,2023-07-13 15:29:00,Extremely likely,NaN,NaN,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.000000,neutral,0.000000,4.0,0,5.0,NaN,NaN,NaN,NaN
2,2023-07-13 15:29:00,Extremely likely,The doctor was very understanding and very kno...,NaN,Earls-Court-Surgery,Brompton-Health-PCN,positive,0.969176,neutral,0.000000,8.0,5,5.0,Patient Respect,NaN,NaN,NaN
3,2023-07-13 15:30:00,Extremely likely,NaN,NaN,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.000000,neutral,0.000000,5.0,5,5.0,NaN,NaN,NaN,NaN
4,2023-07-13 15:31:00,Extremely likely,I was spoken to politely on the phone and a te...,No I was happy with everything,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.756987,positive,0.837488,44.0,6,5.0,Follow-up and Continuity of Care,Patient Respect,NaN,NaN


# Add Categories to ChormaDB

In [5]:
from langchain_community.embeddings.ollama import OllamaEmbeddings

def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [6]:
def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

In [ ]:
!pip install chromadb